In [9]:
import os
import pandas as pd

main_publisher = 'OYC'

script_dir = os.path.dirname(os.path.realpath('__file__'))
path = os.path.join(script_dir, '../Data/' + main_publisher + '/')
df_concepts = pd.read_csv(path + 'concepts_bis.csv', sep = '|')
df_concepts.head()

,OER,Concept,PR
0,0,http://en.wikipedia.org/wiki/Race_and_ethnicit...,0.010138
1,1,http://en.wikipedia.org/wiki/African_Americans,0.006259
2,1,http://en.wikipedia.org/wiki/American_Revoluti...,0.006966
3,1,http://en.wikipedia.org/wiki/Citizenship,0.005502
4,1,"http://en.wikipedia.org/wiki/Concord,_Massachu...",0.018161


In [10]:
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import time

def get_classes(concept):
    classes = []
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setReturnFormat(JSON)
    request = """
                    SELECT ?class WHERE {
            <http://dbpedia.org/resource/CONCEPT> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> ?class
        }
    """
    request = request.replace('CONCEPT', concept)
    sparql.setQuery(request)
    done = False
    while not done : 
        try:
            ret = sparql.queryAndConvert()

            for r in ret["results"]["bindings"]:
                value = r["class"]["value"]
                if ("/yago/" not in value) and ("/umbel/" not in value) and (".wikidata." not in value):
                    classes.append({
                        "Concept" : concept,
                        "Class" : value
                    })
            done = True
            print(f'Concept {concept} done.')
        except Exception as e:
            print('Runtime Error : ', e)
            print('Sleeping for 10 seconds...')
            time.sleep(10)
            print('Sleep done !')
    return classes

In [11]:
y = df_concepts['Concept'].apply(lambda x : get_classes(x.split('/')[-1]))
y = [item for sublist in y for item in sublist]
df_classes = pd.DataFrame(y)
df_classes.head(20)

Concept Race_and_ethnicity_in_the_United_States_census done.
Concept African_Americans done.
Concept American_Revolutionary_War done.
Concept Citizenship done.
Concept Concord,_Massachusetts done.
Concept History_of_the_United_States done.
Concept Massachusetts done.
Concept Minutemen done.
Concept Orthopedic_surgery done.
Concept Race_and_ethnicity_in_the_United_States_census done.
Concept United_States done.
Concept Concord,_Massachusetts done.
Concept John_Jack done.
Concept Slavery done.
Concept United_States done.
Concept Confederate_States_of_America done.
Concept Cotton done.
Concept Credit_card done.
Concept Debit_card done.
Concept JPEG done.
Concept Nation_state done.
Concept Ralph_Ellison done.
Concept Scrip done.
Concept Slavery done.
Concept United_States_one-dollar_bill done.
Concept Barack_Obama done.
Concept Flag_of_Georgia_(U%2eS%2e_state) done.
Concept Flags_of_the_Confederate_States_of_America done.
Concept Laredo,_Texas done.
Concept National_Football_League done.
C

,Concept,Class
0,Race_and_ethnicity_in_the_United_States_census,http://www.w3.org/2002/07/owl#Thing
1,African_Americans,http://www.w3.org/2002/07/owl#Thing
2,African_Americans,http://dbpedia.org/ontology/Band
3,African_Americans,http://dbpedia.org/ontology/EthnicGroup
4,American_Revolutionary_War,http://www.w3.org/2002/07/owl#Thing
5,American_Revolutionary_War,http://schema.org/Event
6,American_Revolutionary_War,http://www.ontologydesignpatterns.org/ont/dul/...
7,American_Revolutionary_War,http://dbpedia.org/ontology/SocietalEvent
8,American_Revolutionary_War,http://dbpedia.org/ontology/Event
9,American_Revolutionary_War,http://dbpedia.org/ontology/MilitaryConflict


In [12]:
df_classes.to_csv(path + "classes.csv", sep = '|')